# Inversions with okapy and SciPy

A simple inversion making use of the Powell algorithm built into the scipy.optimize.minimize function, the Okada model in okapy and our downsampled data in 'okinv' format to try to obtain a best-fitting model of an earthquake.

## Set it up! 

Let's start with some dependencies:

In [1]:
from okapy import rect_shear_fault, los_penalty_fault, test_args
from math import sin, cos, tan, radians, floor
from matplotlib import cm, colors
from scipy.optimize import minimize, Bounds
import numpy as np
import matplotlib.pyplot as plt

## Defining the data, model parameters and elastic constants

Load in the downsampled interfrogram data:

In [2]:
# let's try to make a list!
data = list()
datanames = list()

# load in a data file (need to copy this block for each data file)
# add the full path to the data file here:
indata = np.loadtxt('/home/gareth/Dropbox/quakes/bodrum_turkey_2017/okinv_gpsx10/bdrm_2017_s1_asc.okinv', 
                    usecols=(0,1,2,3,4,5))
indata[:,0]*=1000  # convert x coord from km to m
indata[:,1]*=1000  # convert y coord from km to m
data.append(indata)

# load in a data file (need to copy this block for each data file)
# add the full path to the data file here:
indata = np.loadtxt('/home/gareth/Dropbox/quakes/bodrum_turkey_2017/quadtree_dsc/bdrm_2017_s1_t036_dsc.okinv', 
                    usecols=(0,1,2,3,4,5))
indata[:,0]*=1000  # convert x coord from km to m
indata[:,1]*=1000  # convert y coord from km to m
data.append(indata)

Let's specify some parameters for our models: starting guesses and uncertainties

In [5]:
# for each quantity, starting guess is the first value, sigma the second
strike = [60, 10]        # in degrees
dip = [85, 10]           # in degrees
rake = [1, 20]           # in degrees
slip = [2, 0.5]          # in m
xs = [500000, 5000]        # x coord of center of updip fault projection, in m 
ys = [4240000, 5000]       # y coord of same, in m
as_length = [25000, 2000]     # along-strike fault length, in m
top_depth = [4000, 2000]      # depth of top edge of fault, in m
bottom_depth = [12000, 5000]  # depth to bottom edge of fault

fpstart = np.array([strike[0], dip[0], rake[0], slip[0], xs[0], ys[0], as_length[0], top_depth[0], bottom_depth[0]])
fpsigma = np.array([strike[1], dip[1], rake[1], slip[1], xs[1], ys[1], as_length[1], top_depth[1], bottom_depth[1]])

# let's calculate some 2-sigma bounds on these starting values:
fplowb = fpstart-2*np.array([strike[1], dip[1], rake[1], slip[1], xs[1], ys[1], as_length[1], top_depth[1], bottom_depth[1]])
fphighb = fpstart+2*np.array([strike[1], dip[1], rake[1], slip[1], xs[1], ys[1], as_length[1], top_depth[1], bottom_depth[1]])

# find a random starting model (assume flat pdf between lower and upper bounds)
fparams_restart = fpstart + np.multiply(((np.random.random_sample(9)*4)-2),fpsigma)

# sanity check of depths (make sure bottom depth is greater than top depth)
if fparams_restart[7] > fparams_restart[8]:
    bottom = fparams_restart[7]
    fparams_restart[7] = fparams_restart[8] 
    fparams_restart[8] = bottom
    
# and output the starting model    
print("example starting fault parameters:")
print('  strike:',fparams_restart[0],'dip:',fparams_restart[1],'rake:',fparams_restart[2],'slip:',fparams_restart[3])
print('  xs:',fparams_restart[4],'ys:',fparams_restart[5])
print('  length:',fparams_restart[6],'top:',fparams_restart[7],'bottom:',fparams_restart[8])

example starting fault parameters:
  strike: 62.24921132963169 dip: 92.7064999878689 rake: -27.18068464910332 slip: 1.5760117252802974
  xs: 492619.6571514064 ys: 4244345.1815345185
  length: 23659.163187166796 top: 5861.570855181357 bottom: 16985.10869368022


Finally, let's define some elastic parameters, using some standard values:

In [6]:
eparams = np.array([30e9, 30e9])  # 1st and 2nd Lame elastic parameters; try 30 GPa for both

In [12]:
rundata = data

# and test the new okapy with the new input format

if type(rundata) is list:        
    # if a list, then we shall unpack it 
    print('list!')
            
elif type(rundata) is np.ndarray:
    # if an array, do very little
    print('array!')
        
else:
    print('no idea what this is?!')

    
alldata=np.vstack(rundata)
print('input data, length: {0:d}, total length: {1:d}, numpy size {2:d}'.format(len(rundata),len(alldata),np.size(alldata[:,0])))

modeldisps = rect_shear_fault(fparams_restart, eparams, rundata)

list!
input data, length: 2, total length: 532, numpy size 532


In [15]:
#type(moo)
len(modeldisps[1])

263

## Evaluating the penalty function for a random starting model

We now have all the elements we need to calculate an initial penalty value, using the los_penalty_fault function in okapy. This function has the following syntax: 

<center>penalty=los_penalty_fault(fparams, eparams, data)</center>

The output argument, 'penalty' is the total squared misfit of the model to the data, accounting for a zero-level shift.

There are three input arguments, for which we have already seen examples. 'fparams' is a vector of fault parameters, of the type we have already seen (e.g. fstart, fparams_restart), 'eparams' contains elastic constants, and 'data' is a matrix of data point locations, values and line-of-sight parameters. (You may recognize that these are the same inputs that we used to compute our forward model displacements in the forward modeling notebook...)

For example, we can evaluate the penalty function for our starting model:

In [ ]:
init_penalty = los_penalty_fault(fparams_restart, eparams, data)

print('initial penalty:',init_penalty, 'm^2')

## Optimizing fault parameters using the Powell algorithm in SciPy

The <a href="https://docs.scipy.org/doc/scipy/reference/tutorial/optimize.html#constrained-minimization-of-multivariate-scalar-functions-minimize">'optimize.minimize'</a> function in SciPy is a very flexible optimization package that can apply several different optimization algorithms to the problem of minimizing a specified nonlinear function. Here we want to find the parameter values that minimize our penalty function, 'los_penalty_fault'.

We will make use of the <a href="https://en.wikipedia.org/wiki/Powell%27s_method">Powell algorithm</a>, which is a 'direction set' minimization algorithm that, in our experience, works well for these sorts of problems. It allows us to specify bounds on the inversion parameters, which is particularly useful for limiting the range of models it will test. The Powell algorithm also requires a starting guess, for which we will use our random starting model ('fparams_restart'). 

Note that our 'los_penalty_fault' function has three input arguments &ndash; the fault parameters, the elastic parameters and the data. We only want the Powell algorithm to vary the first of these (which it does by default), but we need to pass all three to properly evaluate the penalty. scipy.optimize allows us to pass the additional arguments to the penalty function using the 'args' option.

In [ ]:
# the scipy.optimize way of setting bounds:
fpbounds = Bounds(fplowb,fphighb)

# and run the Powell algorithm minimizer! 
results = minimize(los_penalty_fault, fparams_restart, args=(eparams, data),  method='Powell', 
                   bounds=fpbounds)

We can look at the results of this optimization run, by printing the contents of 'results':

In [ ]:
results

Of these, 'direc' shows directions in parameter space that the algorithm took; 'fun' is the final, minimum value of the penalty function; 'nfev' is the number of penalty function evaluations (which might explain the delay in getting a result); 'nit' is the number of iterations (direction changes), and 'x' is the final set of fault parameters (that correspond to the minimum penalty, 'fun'). 

We can extract the values of interest here, and hopefully verify that the final penalty is smaller than the one we started with:  

In [ ]:
fparams_out = results.x
output_penalty = results.fun

print("final fault parameters:")
print('  strike:',fparams_out[0],' dip:',fparams_out[1],' rake:',fparams_out[2],' slip:',fparams_out[3])
print('  xs:',fparams_out[4],' ys:',fparams_out[5])
print('  length:',fparams_out[6],' top:',fparams_out[7],' bottom:',fparams_out[8])
print('initial penalty:',init_penalty,'m^2  final penalty:',output_penalty,'m^2')

## Local vs global minima

Technically, what our single run of the Powell algorithm has found is a 'local minimum' of the penalty function. For complicated functions, such as the penalty function of an Okada model fitted to InSAR data that contain noise (our situation!) it is usually the case that there are several, or perhaps many such minima. It's analous to finding the deepest point on the surface of the Moon, by always taking the steepest path downhill. Eventually, you will walk into a crater, and find your way to the bottom of it. But what if two craters over, there is a deeper one?

The problem of finding a 'global minimum' of the penalty function, corresponding to the best-fitting of all possible models, is not straightforward or exact. Following the crater analogy, it can be very dependent on your starting location &ndash; sometimes you need to start away from smaller holes in order to walk into bigger ones! Which is why, when we set up this optimization problem, we randomized our starting guess of the answer. You also have to be careful to specify solution bounds large enough that the neighborhood of the best answer is covered. 

By comparing your answer with your classmates' answers, which originated from different random starting guesses, within a fairly conservative set of bounds,  you should be able to see if there are smaller misfits out there. Or alternatively, if you don't want to crowdsource the problem, you could loop through a large number (say 100) of optimization runs with randomised starting guesses (you will often see these described as 'Monte Carlo restarts'), and retain the best models and penalties from each, in order to find a global minimum solution. 

Paste the best answer you can find here:

In [ ]:
best_fparams=[6.06231351e+01, 1.04999984e+02, 2.59162348e+00, 1.00018395e+00,
       4.99746476e+05, 4.24024712e+06, 2.89999974e+04, 2.11015719e+03,
       1.14709218e+04]
best_penalty=0.11499180622286244

## It is good practice to plot your results and residuals

We can use the routines we developed in earlier notebooks to do this!

In [ ]:
# calculate the displacements
model_los_disps = rect_shear_fault(best_fparams, eparams, data)

# calculate the mean residual
zero_shift = np.mean(data[:,2]-model_los_disps)

# calculate the residual without nuisances...
shifted_data = data[:,2]-zero_shift
residual_los_disps = model_los_disps-shifted_data

# color limits based on the shifted data
cmin, cmax = shifted_data.min(), shifted_data.max()

fig, (ax1,ax2,ax3) = plt.subplots(ncols=3,figsize=(18,8))
axlist = [ax1,ax2,ax3]   # handles for your subplots

# scatter with colormap mapping to z value
scat=ax1.scatter(data[:,0],data[:,1],s=20,c=shifted_data, marker = 'o', cmap = cm.jet, vmin = cmin, vmax = cmax);
ax1.set_xlabel("UTM x (m)",fontsize=12)
ax1.set_ylabel("UTM y (m)",fontsize=12)
ax1.title.set_text('data')
ax1.grid(True,linestyle='-',color='0.75')
ax1.set_aspect('equal')

# scatter with colormap mapping to z value
scat=ax2.scatter(data[:,0],data[:,1],s=20,c=model_los_disps, marker = 'o', cmap = cm.jet, vmin = cmin, vmax = cmax);
ax2.set_xlabel("UTM x (m)",fontsize=12)
ax2.title.set_text('inverse model')
ax2.grid(True,linestyle='-',color='0.75')
ax2.set_aspect('equal')

# scatter with colormap mapping to z value
scat=ax3.scatter(data[:,0],data[:,1],s=20,c=residual_los_disps, marker = 'o', cmap = cm.jet, vmin = cmin, vmax = cmax);
ax3.set_xlabel("UTM x (m)",fontsize=12)
ax3.title.set_text('residual')
ax3.grid(True,linestyle='-',color='0.75')
ax3.set_aspect('equal')

clb=fig.colorbar(scat,ax=axlist)
clb.ax.set_title('LOS disp (m)')

plt.show();